In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/ Image_CVs

/content/drive/MyDrive/ Image_CVs


In [3]:
!pip install -r requirement.txt 

# Library

In [4]:
import matplotlib.pyplot as plt
from PIL import Image

from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
from paddleocr import PaddleOCR

import time
import io
import cv2
import numpy as np
from PIL import Image
import sys
import pandas as pd
import fitz
from tqdm import tqdm

In [5]:
config = Cfg.load_config_from_name('vgg_seq2seq')
config['predictor']['beamsearch']=False
config['device'] = 'cpu'
config['weights'] = './vietocr/transformerocr.pth'
detector = Predictor(config)

In [6]:
ocr = PaddleOCR(use_angle_cls=True, lang='en') # need to run only once to download and load model into memory

Namespace(cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/root/.paddleocr/2.1/cls', cls_thresh=0.9, det=True, det_algorithm='DB', det_db_box_thresh=0.5, det_db_thresh=0.3, det_db_unclip_ratio=1.6, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/root/.paddleocr/2.1/det/en', drop_score=0.5, enable_mkldnn=False, gpu_mem=8000, image_dir='', ir_optim=True, label_list=['0', '180'], lang='en', max_text_length=25, rec=True, rec_algorithm='CRNN', rec_batch_num=6, rec_char_dict_path='./ppocr/utils/en_dict.txt', rec_char_type='ch', rec_image_shape='3, 32, 320', rec_model_dir='/root/.paddleocr/2.1/rec/en', use_angle_cls=True, use_dilation=False, use_gpu=True, use_pdserving=False, use_space_char=True, use_tensorrt=False, use_zero_copy_run=False)


In [7]:
def img2text(img):
  
  # Input: array image
  # Output: a dataframe with bounding box and text

  # Detect line
  result = ocr.ocr(img, rec=False)
  result.reverse()

  # Get bbox
  xs = []
  ys = []
  ws = []
  hs = []
  list_text = []

  # Create df
  for line in result:
      bbox = (line[0][0]-5, line[0][1]-5, line[2][0]+5, line[2][1]+5);
      xs.append(bbox[0])
      ys.append(bbox[1])
      ws.append(bbox[2])
      hs.append(bbox[3])

      new_image = img.crop(bbox)
      list_text.append(detector.predict(new_image))
  bbox_df = pd.DataFrame({'x':xs, 'y':ys, 'w':ws, 'h':hs, 'text': list_text})
  return bbox_df

In [8]:
def img2text(img):
  # Detect line
  result = ocr.ocr(img, rec=False)
  result.reverse()

  # Get bbox
  xs = []
  ys = []
  ws = []
  hs = []
  list_text = []

  img = Image.fromarray(img)
  # Create df
  for line in result:
      bbox = (line[0][0]-5, line[0][1]-5, line[2][0]+5, line[2][1]+5);
      xs.append(bbox[0])
      ys.append(bbox[1])
      ws.append(bbox[2])
      hs.append(bbox[3])
      new_image = img.crop(bbox)
      list_text.append(detector.predict(new_image))
  bbox_df = pd.DataFrame({'x':xs, 'y':ys, 'w':ws, 'h':hs, 'text': list_text})
  return bbox_df

In [9]:
def pix2np(pix):
    if pix.n>3:
        pix = fitz.Pixmap(fitz.csRGB, pix)
    im = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
    return im

In [14]:
def pdfimage2text(file):
  zoom = 1
  mat = fitz.Matrix(zoom, zoom)
  doc = fitz.Document(file)
  pages = doc.pageCount
  block_df = pd.DataFrame()
  for num_page in range(pages):
    pix = doc[num_page].getPixmap(matrix = mat, alpha = False)
    pix = pix2np(pix)
    block_df = pd.concat([block_df, img2text(pix)])
  return block_df

In [15]:
import time
start = time.time()
pdf_path = './cvSample4.pdf'
print(pdfimage2text(pdf_path))
print(time.time()-start)

        x      y  ...      h                                               text
0   140.0   95.0  ...  129.0                                     SƠ YẾU LÝ LỊCH
1    48.0  229.0  ...  253.0                               L/ Thông tin cá nhân
2    48.0  257.0  ...  281.0                          Họ và tên: TRẦN NGỌC DIỄM
3    50.0  287.0  ...  308.0                                      Giới tính: Nữ
4    50.0  313.0  ...  338.0                                Quốc tịch: Việt Nam
..    ...    ...  ...    ...                                                ...
1    82.0   95.0  ...  122.0  Mong muốn có được công việc phù hợp với khả nă...
2    83.0  114.0  ...  141.0  bản thân nhằm đóng góp có hiệu quả vào sự phát...
3    67.0  141.0  ...  171.0         Mong muốn được gắn bó lâu dài với công ty.
4    83.0  174.0  ...  196.0  Môi trường làm việc thân thiện, năng động, chu...
5    84.0  192.0  ...  215.0                                        thăng tiến.

[140 rows x 5 columns]
25.1584384441375